In [6]:
import psutil
import pywinauto
from pywinauto.application import Application
import sys
import pickle

In [5]:
%pip install pywinauto

  Using cached pywinauto-0.6.8-py2.py3-none-any.whl (362 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
PID = 0
for proc in psutil.process_iter():
    try:
        pinfo = proc.as_dict(attrs=['pid', 'name'])
    except psutil.NoSuchProcess:
        pass
    else:
        if 'WeChat.exe' == pinfo['name']:
            PID = pinfo['pid']

In [138]:
app = Application(backend='uia').connect(process=PID)

In [153]:
chat_win = app['小狐狸🦊']
#print(chat_win.dump_tree())
#print("-----------------")
#print(chats)

In [227]:
def search(message):
    msg_contents = message.children()[0].children()[1].children()
    time = msg_contents[0].children()[2].window_text()
    author = msg_contents[0].children()[0].window_text()
    try:
        msg = msg_contents[1].children()[0].children()[0].children()[0].window_text()
    except IndexError as e:
        msg = "[图片]"
    if not msg:
        msg = "[动画表情]"
    return time, author, msg

class RawMessage:
    def __init__(self, time, author, msg):
        self.time = time
        self.author = author
        self.msg = msg

all_msg = []
for cycle in range(0, 50):
    chats = chat_win.wrapper_object().descendants()
    current_round = []
    for message in chats:
        classname = message.friendly_class_name()
        if (classname == "ListItem"):
            time, author, msg = search(message)
            #print("---------")
            #print(time)
            #print(author)
            #print(msg)
            current_round.append(RawMessage(time, author, msg))
    all_msg.extend(current_round)
    cords = chat_win.rectangle()
    pywinauto.mouse.scroll(wheel_dist=4, coords=(cords.left+10, cords.bottom-10))

In [230]:
def checkSame(a, b):
    if a.time == b.time and a.author == b.author and a.msg == b.msg:
        return True
    return False

def checkSeen(l, a):
    for i in l:
        if checkSame(i, a):
            return True
    return False

def printRawMsg(message):
    print(f"{message.time} | {message.author}: {message.msg}")

# Will result in less messages than actual. Fine with me though, I don't know how to process it properly.
seen = set()
uniq = []
for msg in all_msg:
    if not checkSeen(seen, msg):
        uniq.append(msg)
        seen.add(msg)

# all_msg = uniq[::-1]
all_msg = uniq
for msg in all_msg:
    printRawMsg(msg)
    continue
    
# TODO: The order of the messages if fucked up for some reason. I guess we just have to ignore message order within the minute...

昨天 | 小狐狸🦊: traffic不是london吗
昨天 | Andy: [Grimace]
昨天 | Andy: 除了rio我不知道有啥了
昨天 | Andy: 宝宝要不你帮我整理下[Toasted]
昨天 | 小狐狸🦊: 1
昨天 | Andy: oh是吗
昨天 | 小狐狸🦊: 来的*
昨天 | 小狐狸🦊: 你的stats哪来来的
昨天 | 小狐狸🦊: 我看看吧
昨天 | 小狐狸🦊: ok
昨天 | 小狐狸🦊: gpt？
昨天 | Andy: gpt
昨天 | Andy: 数据有问题吗
昨天 | Andy: 应该没有吧？
昨天 | 小狐狸🦊: 我等会看看
昨天 | 小狐狸🦊: 不知道诶
昨天 | Andy: ？
昨天 | 小狐狸🦊: [图片]
昨天 | 小狐狸🦊: btw
昨天 | 小狐狸🦊: 乐的
昨天 | Andy: 荒谬
昨天 | 小狐狸🦊: 哥们儿想冲数学领奖台？？
昨天 | Andy: bevis有竞争对手了
昨天 | 小狐狸🦊: [Doge][Doge][Doge]
昨天 | 小狐狸🦊: karen有竞争对手了
昨天 | Andy: [Doge][Doge][Doge]
昨天 | Andy: 哈哈哈哈哈哈哈哈哈哈哈哈哈哈
昨天 | 小狐狸🦊: 她今天不在学校
昨天 | 小狐狸🦊: 下周吧
昨天 | Andy: 去找igor了吗
昨天 | Andy: karen怎么样
昨天 | 小狐狸🦊: 她肯定上
昨天 | 小狐狸🦊: 不是我说
昨天 | 小狐狸🦊: 我赌karen肯定上
昨天 | Andy: Kinda like ur geo
昨天 | 小狐狸🦊: 真的
昨天 | Andy: 不上就离谱
昨天 | Andy: 她全满的
昨天 | Andy: [动画表情]
昨天 | 小狐狸🦊: [动画表情]
8:02 am | Andy: 我现在去找
8:02 am | 小狐狸🦊: ？
8:02 am | Andy: 收到
8:02 am | 小狐狸🦊: [图片]
8:02 am | Andy: ？
8:02 am | 小狐狸🦊: 不用找了
8:02 am | 小狐狸🦊: 我不会
8:02 am | Andy: 阴间
8:02 am | Andy: 66
8:02 am | 小狐狸🦊: 7m是cbd
8:03 am | 小狐狸🦊: 死
8:03 am | 小狐狸

In [285]:
import datetime

class Message():
    def __init__(self, date, time, author, msg):
        self.date = date
        self.time = time
        self.author = author
        self.msg = msg

current_date = datetime.date.today()

weekdays = {}
eng_to_chinese = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期天'
}
date_format = '%y/%m/%d'

for i in range(2,7):
    delta = datetime.timedelta(days=i)
    date = current_date - delta
    weekname = eng_to_chinese[date.strftime('%A')]
    weekdays[weekname] = date
    
# date = weekdays.get("weekname")
weekdays["今天"] = current_date
weekdays["昨天"] = current_date-datetime.timedelta(days=1)

proc_msg = []

def printMsg(message):
    print(f"{message.date} {message.time} | {message.author}: {message.msg}")

for msg in all_msg:
    time = msg.time
    date = weekdays.get(time)
    if date is not None:
        proc_msg.append(Message(date, "NA", msg.author, msg.msg))
    elif type(time) == str:
        #print(time)
        hour, minute = time.split(' ')[0].split(':')
        hour = int(hour)
        minute = int(minute)
        if time.split(' ')[1] == "pm" and hour != 12:
            hour += 12
        #print(hour, minute)
        #print("---------")
        proc_msg.append(Message(current_date, datetime.time(hour, minute, 0), msg.author, msg.msg))
    else:
        proc_msg.append(Message(time, "NA", msg.author, msg.msg))

for msg in proc_msg:
    printMsg(msg)

2023-05-17 NA | 小狐狸🦊: traffic不是london吗
2023-05-17 NA | Andy: [Grimace]
2023-05-17 NA | Andy: 除了rio我不知道有啥了
2023-05-17 NA | Andy: 宝宝要不你帮我整理下[Toasted]
2023-05-17 NA | 小狐狸🦊: 1
2023-05-17 NA | Andy: oh是吗
2023-05-17 NA | 小狐狸🦊: 来的*
2023-05-17 NA | 小狐狸🦊: 你的stats哪来来的
2023-05-17 NA | 小狐狸🦊: 我看看吧
2023-05-17 NA | 小狐狸🦊: ok
2023-05-17 NA | 小狐狸🦊: gpt？
2023-05-17 NA | Andy: gpt
2023-05-17 NA | Andy: 数据有问题吗
2023-05-17 NA | Andy: 应该没有吧？
2023-05-17 NA | 小狐狸🦊: 我等会看看
2023-05-17 NA | 小狐狸🦊: 不知道诶
2023-05-17 NA | Andy: ？
2023-05-17 NA | 小狐狸🦊: [图片]
2023-05-17 NA | 小狐狸🦊: btw
2023-05-17 NA | 小狐狸🦊: 乐的
2023-05-17 NA | Andy: 荒谬
2023-05-17 NA | 小狐狸🦊: 哥们儿想冲数学领奖台？？
2023-05-17 NA | Andy: bevis有竞争对手了
2023-05-17 NA | 小狐狸🦊: [Doge][Doge][Doge]
2023-05-17 NA | 小狐狸🦊: karen有竞争对手了
2023-05-17 NA | Andy: [Doge][Doge][Doge]
2023-05-17 NA | Andy: 哈哈哈哈哈哈哈哈哈哈哈哈哈哈
2023-05-17 NA | 小狐狸🦊: 她今天不在学校
2023-05-17 NA | 小狐狸🦊: 下周吧
2023-05-17 NA | Andy: 去找igor了吗
2023-05-17 NA | Andy: karen怎么样
2023-05-17 NA | 小狐狸🦊: 她肯定上
2023-05-17 NA | 小狐狸🦊: 不是我说
20

In [288]:
import csv

filename = 'output.csv'

with open(filename, 'w', encoding='gbk', newline='') as file:
    writer = csv.writer(file)

    # Write headers
    headers = ['Date', 'Time', 'Author', 'Msg']
    writer.writerow(headers)
    
    for msg in proc_msg:
        writer.writerow([msg.date, msg.time, msg.author, msg.msg])

print(f"Data exported to {filename} successfully.")

UnicodeEncodeError: 'gbk' codec can't encode character '\U0001f98a' in position 17: illegal multibyte sequence